In [1]:
import pandas as pd


Importing ERP data as Data Frame for Wrangling

In [2]:
df = pd.read_excel("/content/DynamicsExport_637717296556323714.xlsx")
df.head

<bound method NDFrame.head of      Priority  Overdue  Work cell       Kanban  Card number Item number  \
0          No    False         48  KN-00086009         2312  8688-00001   
1          No    False         48  KN-00138571         3506  8687-00010   
2          No    False         48  KN-00135866         2858  8680-01348   
3          No    False         48  KN-00135868         2860  8680-00155   
4          No    False         48  KN-00135869         2861  8680-00157   
...       ...      ...        ...          ...          ...         ...   
1764       No    False         48  KN-00361947        35924  8640-01119   
1765       No    False         48  KN-00364496        36289  8680-01012   
1766       No    False         48  KN-00359490        35649  8680-01003   
1767       No    False         48  KN-00359491        35650  8680-01003   
1768       No    False         48  KN-00362503        36012  8680-01019   

      Job quantity Job status Size Handling unit status  ...  \
0    

In [3]:
print(df.info())
print(df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1769 entries, 0 to 1768
Data columns (total 42 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Priority                   1769 non-null   object        
 1   Overdue                    1769 non-null   bool          
 2   Work cell                  1769 non-null   int64         
 3   Kanban                     1769 non-null   object        
 4   Card number                1769 non-null   int64         
 5   Item number                1769 non-null   object        
 6   Job quantity               1769 non-null   int64         
 7   Job status                 1769 non-null   object        
 8   Size                       1769 non-null   object        
 9   Handling unit status       1769 non-null   object        
 10  Planned period             1769 non-null   datetime64[ns]
 11  Ship date                  1759 non-null   datetime64[ns]
 12  Due da

Treating Missing values:

In [4]:
print("DF Columns:")
print(df.columns.tolist())

DF Columns:
['Priority', 'Overdue', 'Work cell', 'Kanban', 'Card number', 'Item number', 'Job quantity', 'Job status', 'Size', 'Handling unit status', 'Planned period', 'Ship date', 'Due date/time', 'Completed by', 'Completed DateTime', 'Actual prepared date/time', 'Actual start date/time', 'Actual end date/time', 'Name', 'Schedule group', 'Warehouse', 'Location', 'Good quantity', 'Error quantity', 'Sales order', 'Line number', 'Customer', 'Delivery name', 'External', 'Line status', 'Requested ship date', 'Confirmed ship date', 'Deliver remainder', 'Unit price', 'Price unit', 'Net amount', 'Sales group', 'Created by', 'Created date and time', 'Modified by', 'Modified date and time', 'Expected start date/time']


In [5]:
missing_value = df.isnull().sum()
print(missing_value)

Priority                       0
Overdue                        0
Work cell                      0
Kanban                         0
Card number                    0
Item number                    0
Job quantity                   0
Job status                     0
Size                           0
Handling unit status           0
Planned period                 0
Ship date                     10
Due date/time                  0
Completed by                 505
Completed DateTime           505
Actual prepared date/time      0
Actual start date/time         0
Actual end date/time           0
Name                           0
Schedule group                 9
Warehouse                      0
Location                       0
Good quantity                  0
Error quantity                 0
Sales order                   10
Line number                    0
Customer                      10
Delivery name                 10
External                     338
Line status                   10
Requested 

Filled "Ship Date" with forward fill value:

In [6]:
df['Ship date']
df['Ship date'].fillna(method='ffill', inplace=True)
print(df['Ship date'].isnull().sum())

0


Filled "Completed by" with the mode value:

In [7]:
mode_completed_by = df['Completed by'].mode().iloc[0]
df['Completed by'].fillna(mode_completed_by,inplace=True)
print(df['Completed by'].isnull().sum())

0


Filled "Completed DateTime" value with "Actual prepared date/time":

In [8]:
df['Completed DateTime'].fillna(df['Actual prepared date/time'], inplace=True)
print(df['Completed DateTime'].isnull().sum())

0


Filled "Schedule group" with forward fill value:

In [9]:
df['Schedule group'].fillna(method='ffill', inplace=True)
print(df['Schedule group'].isnull().sum())

0


Removed "Sale Order" and "Customer" NAs:

In [10]:
df.dropna(subset=['Sales order'], inplace=True)
print(df['Sales order'].isnull().sum())
df.dropna(subset=['Customer'], inplace=True)
df.dropna(subset=['Confirmed ship date'], inplace=True)
df.dropna(subset=['Sales group'], inplace=True)
print(df['Sales group'].isnull().sum())
print(df['Confirmed ship date'].isnull().sum())

0
0
0


Filling Na values with "Unknown":

In [11]:
df["External"].fillna("Unknown", inplace=True)
print(df['External'].isnull().sum())

0


In [12]:
df['Sales group'] = pd.to_numeric(df['Sales group'], errors='coerce')
df['Due date/time'] = df['Due date/time'].astype(str)
df[['Due Date', ' Due Time']] = df['Due date/time'].str.split(' ', expand=True)


Changing Date and Time column to two seprate columns: Loop through columns that has both date & time and droped time column since it wasn't desirable for analysis.

In [ ]:
df['Due date/time'] = df['Due date/time'].astype(str)
df[['Due Date', 'Due Time']] = df['Due date/time'].str.split(' ', expand=True)
df.drop(['Due Time'], axis=1, inplace=True)
print(df.columns)


In [25]:
datecols= ['Completed DateTime', 'Actual prepared date/time','Actual start date/time', 'Actual end date/time', 'Created date and time', 'Modified date and time', 'Expected start date/time' ]

for item in datecols:
  if item in df.columns:
    df[item] = df[item].astype(str)
    df[[item +'Date', item+'Time']] = df[item].str.split(' ', expand=True)
    df.drop([item+'Time'], axis=1, inplace=True)


In [30]:
df.to_excel('ERP_data.xlsx', index=False)

# New Section